# Designing & Creating a Database

In this project I will work with a file from [Major League Baseball](https://en.wikipedia.org/wiki/Major_League_Baseball) games from [Retrosheet](www.retrosheet.org). The goal of the project is to:

* Import data into SQLite
* Design a normalized database schema
* Create tables for our schema
* Insert data into our schema

Retrosheet compiles detailed statistics on baseball games from the 1800s through to today. The main file we will be working from game_log.csv, has been produced by combining 127 separate CSV files from retrosheet, and has been pre-cleaned to remove some inconsistencies. The game log has hundreds of data points on each game which we will normalize into several separate tables using SQL, providing a robust database of game-level statistics.

Since we are  trying to create a normalized database, so our focus should be:

* Becoming familiar, at a high level, with the meaning of each column in each file.
* Thinking about the relationships between columns within each file.
* Thinking about the relationships between columns across different files.

**Disclaimer:** This project is prepared as part of the guided projects on [dataquest](http://dataquest.io/).

## Data Exploration

Setting the below options after we import pandas is recommended– they will prevent the DataFrame output from being truncated, given the size of the main game log file. Let's also read in the data and explore it. To better understand columns we can use the following [game_log_fields.txt](data/game_log_fields.txt) file

In [ ]:
# load libs
import pandas as pd
import sqlite3

# set pandas options
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

# read dataset
game_log = pd.read_csv('data/game_log.csv')
park_codes = pd.read_csv('data/park_codes.csv')
person_codes = pd.read_csv('data/person_codes.csv')
team_codes = pd.read_csv('data/team_codes.csv')

In [ ]:
print("Game Log", game_log.shape)
print("Park Codes", park_codes.shape)
print("Person Codes", person_codes.shape)
print("Team Codes", team_codes.shape)

In [ ]:
game_log.head()

Let's explore the above dataset, particularly let's look at what defensive position each number represents. We can observe that columns such `h_player_1_def_pos` and `v_player_9_def_pos` indicate the defensive positions of home player 1 and visiting player 9 respectively. The defensive positions are numbered (1-9). These are the defensive positions with their respective codes [(source)](https://en.wikipedia.org/wiki/Baseball_positions):

Code. Position
1. Pitcher
2. Catcher
3. First Baseman
4. Second Baseman
5. Third Baseman
6. Shortstop
7. Left Fielder
8. Center Fielder
9. Right Fielder

The image below nicely visualizes these positions.

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Baseball_positions.svg/300px-Baseball_positions.svg.png'>

---
Let's explore the league information. The columns 4-5, 7-8 indicate visiting team & home team leagues. We will look at those values.

In [ ]:
game_log.h_league.value_counts()

In [ ]:
game_log.v_league.value_counts()

We can observe the list of leagues and also notice that majority of games don't have information about their team leagues. The list of leagues and their interpretation:

* AL - American League
* AA - Double A League
* FL - Florida State League
* PL - Players League
* UA - Union Association

In [ ]:
park_codes.head()

In [ ]:
person_codes.head()

In [ ]:
team_codes.head()

## Importing Data into Sqlite

To insert data into a normalized database we need to come up with a primary key for the game log table. Exploring the [Retrosheet site](https://www.retrosheet.org/eventfile.htm), we can find this data dictionary for their event files, which list every event within each game. This includes the following description:

*__id__: Each game begins with a twelve character ID record which identifies the date, home team, and number of the game. For example, ATL198304080 should be read as follows. The first three characters identify the home team (the Braves). The next four are the year (1983). The next two are the month (April) using the standard numeric notation, 04, followed by the day (08). The last digit indicates if this is a single game (0), first game (1) or second game (2) if more than one game is played during a day, usually a double header The id record starts the description of a game thus ending the description of the preceding game in the file.*

This is what we essentially need, where for our primary key we will use a composite key which has been described above. The key uses `date`, `home team` and `number of the game` to make up they composite key.

Our next task is to import the data into SQLite. We will use pandas `DataFrame.to_sql()` function to do that.

In [ ]:
# helper functions
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)
    
def run_command(q):
    with sqlite3.connect('mlb.db') as conn:
        conn.isolation_level = None
        conn.execute(q)

# show the tables
def show_tables():
    q = """
        SELECT name, type
        FROM sqlite_master
        WHERE type IN ("table","view");
    """
    return run_query(q)

In [ ]:
# import dataframe into sqlite database
with sqlite3.connect('mlb.db') as conn:
    game_log.to_sql('game_log', conn, if_exists='replace', index=False)
    park_codes.to_sql('park_codes', conn, if_exists='replace', index=False)
    person_codes.to_sql('person_codes', conn, if_exists='replace', index=False)
    team_codes.to_sql('team_sql', conn, if_exists='replace', index=False)

In [ ]:
# check tables
show_tables()

We will create a new column in `game_log` table called `game_id` which will using the key we discussed above. Composite key - `date`, `home team` and `number of the game`.

In [ ]:
q = 'SELECT * FROM game_log LIMIT 5'
run_query(q)

In [ ]:
# query to add the new column
q = 'ALTER TABLE game_log ADD COLUMN game_id VARCHAR'
run_command(q)

In [ ]:
# query to populate the column with concatenation
q = 'UPDATE game_log SET game_id = h_name || date || number_of_game'
run_command(q)

In [ ]:
# check to see the results
q = 'SELECT * FROM game_log LIMIT 5'
run_query(q)

## Looking for Normalization Opportunities

Through investigation of the tables above, we can spot multiple opportunities where we can normalize our data and eventually the database.

__Repetition in Columns:__

We can see in the below following segment of data that player information is spread out across columns (`id`, `name`, `off_pos`, `def_pos`). We can normalize this information by simply having a separate table with players.

In [ ]:
# check the above mentioned fragment
q = '''
SELECT v_player_1_id, v_player_1_name, v_player_1_def_pos,
        v_player_2_id, v_player_2_name, v_player_2_def_pos
FROM game_log LIMIT 10
'''
run_query(q)

To normalize the data we can convert into a table such as this.

|id|name|def_pos|off_pos
|---|---|---|---|
|villj001|Jonathan Villar|5.0|1.0|
|granc001|Curtis Granderson|8.0|1.0|
|kendh001|Howie Kendrick|7.0|1.0|
|jasoj001|John Jaso|3.0|1.0|
|gordd002|Dee Gordon|4.0|1.0|
|genns001|Scooter Gennett|4.0|2.0|
|cabra002|Asdrubal Cabrera|6.0|2.0|
|turnj001|Justin Turner|5.0|2.0|
|polag001|Gregory Polanco|9.0|2.0|
|telit001|Tomas Telis|2.0|2.0|

We could transfer these into a new table from our `game_log` table but actually our `person_codes` table already contains the `id` and `name` of players. We could remove player name from our `game_log` table since we have player id's in the `person_codes` table.

A similar to above approach could be used across our `game_log` and we can remove the following columns and simply keep player ids associated usually preceding these columns:

* `hp_umpire_name`
* `1b_umpire_name`
* `2b_umpire_name`
* `3b_umpire_name`
* `lf_umpire_name`
* `rf_umpire_name`
* `v_manager_name`
* `h_manager_name`
* `winning_pitcher_name`
* `losing_pitcher_name`
* `saving_pitcher_name`
* `winning_rbi_batter_id_name`
* `v_starting_pitcher_name`
* `h_starting_pitcher_name`

And as discussed above all `v_player_{num}_name` and `h_player_{num}_name` columns would also be removed and associated id's kept.

In [ ]:
q = 'PRAGMA table_info(game_log);'
# run_query(q)

__Redundant Data__

We want to ensure that our database doesn't contain duplicate information, that is data which we can either find in another table or derive. One of those examples can be found in the `park_codes`.  We can check out the first few rows of the `park_codes` table.

In [ ]:
q = 'SELECT * FROM park_codes LIMIT 5'
run_query(q)

The start and end columns show the first and last games played at the park, however we will be able to derive this information by looking at the park information for each game. Similarly, the league information is going to be available elsewhere in our database.

## Planning a Normalized Schema

In this section, we plan to prepare a database schema for our new database. We will use the [DbDesigner](https://www.dbdesigner.net/) tool to design a new schema. Below is the image of our schema.